In [9]:
import pandas as pd
import datetime
from openpyxl import *
from openpyxl.styles import Font
from openpyxl.chart import *

In [2]:
excel_df = pd.read_excel('sales_data_sample.xlsx')
excel_df = excel_df[['QUANTITYORDERED', 'PRICEEACH', 'ORDERDATE', 'PRODUCTLINE', 'PHONE', 'COUNTRY']]

In [3]:
# Create formula columns, can edit directly with openpyxl but this is more intuitive & no need to remember cells
excel_df['TOTALPRICE'] = excel_df['QUANTITYORDERED'] * excel_df['PRICEEACH']
excel_df['ORDERDATE'] = pd.to_datetime(excel_df['ORDERDATE'])
excel_df['MONTH_ID'] = excel_df['ORDERDATE'].dt.month
excel_df['YEAR_ID'] = excel_df['ORDERDATE'].dt.year
excel_df

,QUANTITYORDERED,PRICEEACH,ORDERDATE,PRODUCTLINE,PHONE,COUNTRY,TOTALPRICE,MONTH_ID,YEAR_ID
0,30,95.70,2003-02-24,Motorcycles,2125557818,USA,2871.00,2,2003
1,34,81.35,2003-07-05,Motorcycles,26.47.1555,France,2765.90,7,2003
2,41,94.74,2003-01-07,Motorcycles,+33 1 46 62 7555,France,3884.34,1,2003
3,45,83.26,2003-08-25,Motorcycles,6265557265,USA,3746.70,8,2003
4,49,100.00,2003-10-10,Motorcycles,6505551386,USA,4900.00,10,2003
...,...,...,...,...,...,...,...,...,...
2818,20,100.00,2004-02-12,Ships,(91) 555 94 44,Spain,2000.00,2,2004
2819,29,100.00,2005-01-31,Ships,981-443655,Finland,2900.00,1,2005
2820,43,100.00,2005-01-03,Ships,(91) 555 94 44,Spain,4300.00,1,2005
2821,34,62.24,2005-03-28,Ships,61.77.6555,France,2116.16,3,2005


In [4]:
yearly_sales = excel_df.pivot_table(columns='YEAR_ID', values='TOTALPRICE',aggfunc='sum')
yearly_sales

YEAR_ID,2003,2004,2005
TOTALPRICE,2898149.94,3913700.87,1479035.98


In [5]:
monthly_sales = excel_df.pivot_table(index = 'YEAR_ID', columns='MONTH_ID', values='TOTALPRICE',aggfunc='sum')
monthly_sales

MONTH_ID,1,2,3,4,5,6,7,8,9,10,11,12
YEAR_ID,,,,,,,,,,,,
2003,187385.56,215776.28,189992.19,218214.34,266581.42,217490.48,104289.90,180718.75,170129.13,402610.24,597356.59,147605.06
2004,335535.35,452996.36,152621.35,241597.90,193953.37,271048.60,291909.22,385414.86,224877.78,547302.09,633655.14,182788.85
2005,277798.40,179634.65,311249.70,146126.30,221817.46,90353.67,21944.80,35555.91,58607.32,100185.87,NaN,35761.90


In [6]:
ytd_sales_by_country = excel_df[excel_df.YEAR_ID == excel_df.YEAR_ID.max()].pivot_table(index='COUNTRY', columns='YEAR_ID', values='TOTALPRICE',aggfunc='sum')
ytd_sales_by_country

YEAR_ID,2005
COUNTRY,
Australia,120980.41
Austria,54814.60
Belgium,20111.95
Canada,26262.77
Denmark,17400.00
Finland,110781.32
France,205784.16
Italy,37493.17
Japan,30740.11


In [7]:
ytd_monthly_orders = excel_df[excel_df.YEAR_ID == excel_df.YEAR_ID.max()].pivot_table(index='MONTH_ID',
                                                                                        values='PHONE',
                                                                                        aggfunc='count')
ytd_monthly_orders

,PHONE
MONTH_ID,
1,96
2,61
3,96
4,39
5,71
6,30
7,6
8,10
9,21


In [8]:
ytd_usa_monthly_sales = excel_df[(excel_df.COUNTRY == 'USA') & (excel_df.YEAR_ID == excel_df.YEAR_ID.max())].pivot_table(columns='PRODUCTLINE',
                                                                                                index='MONTH_ID',
                                                                                                values='TOTALPRICE',
                                                                                                aggfunc='sum')
ytd_usa_monthly_sales

PRODUCTLINE,Classic Cars,Motorcycles,Planes,Ships,Trains,Trucks and Buses,Vintage Cars
MONTH_ID,,,,,,,
1,40694.83,NaN,6002.24,6284.00,2986.50,10393.59,34493.62
2,25106.46,NaN,NaN,9088.62,NaN,23836.26,31217.34
3,3300.00,16554.79,44292.22,7500.13,2154.00,NaN,26396.35
4,68616.21,12110.61,NaN,NaN,NaN,NaN,16291.99
5,15400.00,10388.41,NaN,NaN,4449.39,NaN,11494.80
6,1900.00,NaN,NaN,22266.72,4800.00,NaN,15199.49
7,2611.80,6600.00,NaN,NaN,NaN,NaN,NaN
8,3500.00,NaN,NaN,NaN,NaN,NaN,NaN
12,17440.47,NaN,NaN,NaN,NaN,NaN,18321.43


In [11]:
def create_chart(wb_filename, sheet, chart_type, subtype=None, grouping=None, chart_title=None, title_font=None,
                 x_title=None, y_title=None, label_font = None, style=None, overlap=None, hide_legend=False):

        wb = load_workbook(filename=wb_filename)
        
        sheet = wb[sheet]

        # Declaring table range
        min_column = sheet.min_column
        max_column = sheet.max_column
        min_row = sheet.min_row
        max_row = sheet.max_row

        # Dictionary for initializing chart type
        CHART_DICT = {'area': lambda: AreaChart(),
                      'bar': lambda: BarChart(),
                      'bubble': lambda: BubbleChart(),
                      'line': lambda: LineChart(),
                      'scatter': lambda: ScatterChart(),
                      'pie': lambda: PieChart(),
                      'donut': lambda: DoughnutChart(),
                      'radar': lambda: RadarChart(),
                      'stock': lambda: StockChart(),
                      'surface': lambda: SurfaceChart()
        }

        # Adding charts
        chart = CHART_DICT[chart_type]()

        # Set chart subtype & grouping if specified
        if subtype:
                chart.type = subtype
        if grouping:
                chart.grouping = grouping

        # Set title & subtitles, and fonts if specified
        if chart_title:
                chart.title = chart_title
        if title_font:
                chart.title.font = title_font
        if x_title:
                chart.x_axis.title = x_title
        if y_title:
                chart.y_axis.title = y_title
        if label_font:
                chart.x_axis.title.font = label_font
                chart.y_axis.title.font = label_font

        # Set chart style & overlap if specified
        if style:
                chart.style = style
        if overlap:
                chart.overlap = overlap

        # Hide legend if specified
        if hide_legend:
                chart.legend = None
        
        # Hide grid lines
        if chart_type not in ['donut', 'pie']:
                chart.x_axis.majorGridlines = None
                chart.y_axis.majorGridlines = None
                
        data = Reference(sheet,
                        min_col=min_column+1,
                        max_col=max_column,
                        min_row=min_row,
                        max_row=max_row) # including headers
        categories = Reference(sheet,
                            min_col=min_column,
                            max_col=min_column,
                            min_row=min_row+1,
                            max_row=max_row) # not including headers
        
        chart.add_data(data, titles_from_data=True)
        chart.set_categories(categories)
        
        sheet.add_chart(chart, "A15") # Chart placement

        wb.save(wb_filename)

In [13]:
today = datetime.datetime.today().strftime('%d%m%y')

with pd.ExcelWriter(f"Report_{today}.xlsx") as writer:
    yearly_sales.to_excel(writer, sheet_name="Yearly Sales")
    monthly_sales.to_excel(writer, sheet_name="Monthly Sales")
    ytd_monthly_orders.to_excel(writer, sheet_name="YTD Monthly Orders")
    ytd_usa_monthly_sales.to_excel(writer, sheet_name="YTD USA Monthly Sales")

sheet_charts = [["Yearly Sales", "bar", ],
              ["Monthly Sales", "line"],
              ["YTD Monthly Orders", "bar"],
              ["YTD USA Monthly Sales", "donut"]]

# Universal font
title_font = Font(name='Roboto', size=16, bold=True)
label_font = Font(name='Roboto', size=12, italic=True)

create_chart(f"Report_{today}.xlsx", "Yearly Sales", chart_type="bar", chart_title="Yearly Sales", x_title="Year", title_font=title_font, label_font=label_font, y_title="Sales", style=10, overlap=-50, hide_legend=True)
create_chart(f"Report_{today}.xlsx", "Monthly Sales", chart_type="bar", grouping="stacked", chart_title="Monthly Sales", x_title="Month", title_font=title_font, label_font=label_font, y_title="Sales", style=10, overlap=100)
create_chart(f"Report_{today}.xlsx", "YTD Monthly Orders", chart_type="line", chart_title="YTD Monthly Orders", x_title="Month", title_font=title_font, label_font=label_font, y_title="Orders", style=1, hide_legend=True)
create_chart(f"Report_{today}.xlsx", "YTD USA Monthly Sales", chart_type="bar", subtype="col", grouping="stacked", chart_title="YTD USA Monthly Sales", title_font=title_font, label_font=label_font, x_title="Month", y_title="Customers", style=5, overlap=100)